## Can a `vectorize` function call a `jit` function?

In [ ]:
import numpy
from numba import vectorize, jit, njit

In [ ]:
@jit
def add(a, b):
    return a + b

In [ ]:
@vectorize
def add_vec(a, b):
    return add(a, b)

In [ ]:
to_add = numpy.arange(100).reshape(50, 2)

In [ ]:
add_vec(to_add[:, 0], to_add[:, 1])

## Can a `jit` function call a `vectorize` function?

In [ ]:
@vectorize
def add(a, b):
    return a + b

@jit
def add_vec(a, b):
    return add(a, b)

In [ ]:
add_vec(to_add[:, 0], to_add[:, 1])

## Can I use `isinstance` in `jit`ted functions?  

No, but there is a solution, `generated_jit`

Imagine you have a function that takes an array but wants the scalar sum of that array to operate on.  You can just use `.sum()`.  But sometimes, instead of an array getting passed in, a scalar gets passed, which will throw an error since scalars don't have a `sum` method.  

In [ ]:
from numba import types, generated_jit

In [ ]:
@generated_jit
def safesum(M):
    if isinstance(M, types.Array):
        return lambda M: M.sum()
    else:
        return lambda M: M

In [ ]:
safesum(to_add)

In [ ]:
safesum(5)

### Note:

* `safesum` gets called with the Numba types of the argument, not their values
* it returns a _function_, not a value.  

You could use this to have different functions called depending on the type of the inputs

In [ ]:
def add(a, b):
    return a + b

add_nums = jit(nopython=True)(add)
add_str = jit()(add)

In [ ]:
@generated_jit
def safeadd(a, b):
    if isinstance(a, types.Opaque):
        return add_str
    else:
        return add_nums

In [ ]:
safeadd('3', '4')

In [ ]:
safeadd(3, 4)

# Tips

Set envvar `NUMBA_DISABLE_JIT=1` to disable numba compilation (for debugging)

Install the "Hide Traceback" extension if you're prototyping in a notebook.

# Errors

### Unification errors

Thanks to Graham Markhall for the idea for these examples:
http://gmarkall.github.io/tutorials/pycon-uk-2015

When Numba compiles a function just-in-time, it needs to declare the type of the output(s).  If it can't do that in a consistent way, it gets upset.

In [ ]:
@jit(nopython=True)
def get_low(a, b, c):
    if c:
        return a
    else:
        return b

In [ ]:
get_low(3., (3, 2), True)

In [ ]:
get_low(4, numpy.zeros(3), True)

In [ ]:
get_low(3, 4, True)

In [ ]:
@jit(nopython=True)
def dont_index_a_scalar(a):
    return a[0]

In [ ]:
dont_index_a_scalar(5.)

## Globals are treated as compile-time constants by Numba

In [ ]:
a = 5

In [ ]:
@njit
def add_to_a(b):
    return a + b

In [ ]:
add_to_a(7)

In [ ]:
a = 12

In [ ]:
add_to_a(7)

### Solution: Don't use globals(!)

Seriously.  Don't.  But if you must, you can force a recompile of the jitted function.

In [ ]:
add_to_a.recompile()

In [ ]:
add_to_a(7)